In [1]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt

from utils.helpers import *
from utils.plotting import *
from utils.glm_models import *

# Import relevant SBL modules
import sbl_agents.sbl_cat_dir as sbl_cd
import sbl_agents.sbl_hmm as sbl_hmm

# Set random seed for replicability and define directories
np.random.seed(seed=1234)
results_dir = os.getcwd() + "/results/"
fig_dir = os.getcwd() + "/figures/"
data_dir = os.getcwd() + "/data/"

In [2]:
# General Model Settings
model_types = ["SP", "AP", "TP"]
save_results = True
verbose = True

sample_files = [["sub-01/sub-01_ses-1_run-1", "sub-01/sub-01_ses-1_run-2",
                 "sub-01/sub-01_ses-1_run-3", "sub-01/sub-01_ses-1_run-4",
                 "sub-01/sub-01_ses-1_run-5"]]

subject_list = range(len(sample_files))

eeg_files = ["sub-01/sub-01_sbl"]

elec_of_interest = {"FCz": 47, "FC2": 46, "FC4": 45, 
                    "Cz": 48, "C2": 49, "C4": 50,
                    "C6": 51, "CPz": 32, "CP2": 56,
                    "CP4": 55, "CP6": 54}

trial_coding_lookup = {11: "First Regime - Low Intensity",
                       12: "First Regime - High Intensity",
                       21: "Second Regime - Low Intensity",
                       22: "Second Regime - High Intensity",
                       33: "Catch Trial"}

# Select block and electrode for analysis
sampling_rate = 0.3
inter_stim_interval = np.array([-0.05, 0.65])
reg_model_type = "OLS"
template = "Subject {} | Block {} | Electrode {} | Regressor: {} | Time: {:.2f}"

**Loops to Run**
- ~15 Subjects, 5 blocks, ~10 Electrodes
- ~30 Different Models, ~359 Sample Point per event

**TODO**
- [x] HDF5 logging
- [x] Write function that processes the output hdf5 array (average over blocks, sum over subjects)
- [ ] Figure out why null model is so good ;)
- [ ] Check if $\tau$ is working
- [ ] Get CS running
- [ ] Get HMM running
- [ ] Write a general function to get regressors
- [ ] Make everything command line executable

In [6]:
# Subset for proto-typing
sampling_rate = 0.1
inter_stim_interval = np.array([0.1, 0.5])
eoi = {"Cz": 48, "C2": 49, "C4": 50}

In [7]:
# Create Logging object
log = ExperimentLog(num_subjects=1, num_blocks=5,
                    elec_of_interest=eoi,
                    save_fname=results_dir + "trial_by_trial.hdf5")

In [8]:
# Loop over SUBJECTS
for subject_id in subject_list:
    # print("Subject {}: LME Trial-by-Trial Analysis".format(subject_id + 1))
    eeg_data = sio.loadmat("data/" + eeg_files[subject_id] + ".mat")
    
    # Loop over BLOCKS
    for block_id in range(len(sample_files[subject_id])):
        # print("\t Block {}: Load Seq, Comp Regressors".format(block_id + 1))
        
        # Load in the specific trial/stimuli sequence
        sample, meta = load_obj("data/" + sample_files[subject_id][block_id] + ".mat")
        seq, hidden = sample[:, 2], sample[:, 1]
    
        # Compute Surprise Regressors
        CD_PS_SP, CD_BS_SP, CD_CS_SP = sbl_cd.main(seq, hidden, tau=0, model_type="SP")
        CD_PS_AP, CD_BS_AP, CD_CS_AP = sbl_cd.main(seq, hidden, tau=0, model_type="AP")
        CD_PS_TP, CD_BS_TP, CD_CS_TP = sbl_cd.main(seq, hidden, tau=0, model_type="TP")

        # regressors = {"PS-SP": PS_SP, "BS-SP": BS_SP, "CS-SP": CS_SP,
        #               "PS-AP": PS_AP, "BS-AP": BS_AP, "CS-AP": CS_AP,
        #               "PS-TP": PS_TP, "BS-TP": BS_TP, "CS-TP": CS_TP}

        regressors = {"CD_PS_SP": CD_PS_SP, "CD_BS_SP": CD_BS_SP,
                      "CD_PS_AP": CD_PS_AP, "CD_BS_AP": CD_BS_AP,
                      "CD_PS_TP": CD_PS_TP, "CD_BS_TP": CD_BS_TP}
        
        # Loop over ELECTRODES OF INTEREST
        for elec_name, elec_id in eoi.items():
            # print("\t \t Electrode {}: Select EEG".format(elec_name))
            
            # Get the block- and electrode-specific eeg data
            y_elec, y_tw = get_electrode_data(eeg_data, block_id, elec_id,
                                              inter_stim_interval, sampling_rate,
                                              verbose=False)
            # Get null model once for a block 
            # (PS-AP as filler - parallelize does not work with None)
            start = time.time()
            null_model_lme = parallelize_over_samples(y_elec,
                                                      regressor=regressors["CD_PS_AP"],
                                                      reg_model_type="Null")
            t_time = time.time() - start
            
            log.dump_data(subject_id, block_id, elec_name, "Null", null_model_lme)
            log.dump_data(subject_id, block_id, elec_name, "Sample_Points", y_tw)
            print(template.format(subject_id+1, block_id+1, elec_name, "Null", t_time))
           
            # Loop over Different SURPRISE REGRESSORS
            for regressor_type, regressor in regressors.items():
                # print("\t \t \t Model {}: Run in Parallel".format(regressor_type))
                
                start = time.time()
                results = parallelize_over_samples(y_elec, regressor, reg_model_type)
                t_time = time.time() - start
                
                log.dump_data(subject_id, block_id, elec_name, regressor_type, results)
                print(template.format(subject_id+1, block_id+1, elec_name, regressor_type, t_time))

Subject 1 | Block 1 | Electrode Cz | Regressor: Null | Time: 82.66
Subject 1 | Block 1 | Electrode Cz | Regressor: CD_PS_SP | Time: 89.27
Subject 1 | Block 1 | Electrode Cz | Regressor: CD_BS_SP | Time: 91.42
Subject 1 | Block 1 | Electrode Cz | Regressor: CD_PS_AP | Time: 98.18
Subject 1 | Block 1 | Electrode Cz | Regressor: CD_BS_AP | Time: 78.84
Subject 1 | Block 1 | Electrode Cz | Regressor: CD_PS_TP | Time: 85.43
Subject 1 | Block 1 | Electrode Cz | Regressor: CD_BS_TP | Time: 90.26
Subject 1 | Block 1 | Electrode C2 | Regressor: Null | Time: 76.50
Subject 1 | Block 1 | Electrode C2 | Regressor: CD_PS_SP | Time: 100.52
Subject 1 | Block 1 | Electrode C2 | Regressor: CD_BS_SP | Time: 89.62
Subject 1 | Block 1 | Electrode C2 | Regressor: CD_PS_AP | Time: 93.63
Subject 1 | Block 1 | Electrode C2 | Regressor: CD_BS_AP | Time: 87.05
Subject 1 | Block 1 | Electrode C2 | Regressor: CD_PS_TP | Time: 100.41
Subject 1 | Block 1 | Electrode C2 | Regressor: CD_BS_TP | Time: 96.13
Subject 1 | 

TypeError: exceptions must derive from BaseException

In [70]:
import h5py
import tables

def process_tbt_logs(results_dir, num_blocks, log_fnames, elec_name):
    """
    Load in all hdf5 log files (one per subject)
    """
    # Get time frame of analysis
    h5f = tables.open_file(results_dir + log_fnames[0], mode="a")
    g_tw = "subject_0/block_0/elec_Cz/Sample_Points"
    h5f.root[g_tw]
    h5f.close()
    
    # Create empty results place-holder (for all subjects summed)
    results_all = {key: np.zeros((20,)) for key in regressor_names}

    for i, log in enumerate(log_fnames):
        h5f = tables.open_file(results_dir + log, mode="a")
        g_sub = "subject_" + str(i)
        
        # Create empty results place-holder for single subject
        results_temp = {key: [] for key in regressor_names}

        for j in range(num_blocks):
            # Loop over blocks and and regressors - temporarily store lmes
            g_block = g_sub + "/block_" + str(j) + "/elec_" + elec_name
            for reg in regressor_names:  
                out = [t for t in h5f.root[g_block + "/" + reg]]
                results_temp[reg].append(out)
        h5f.close()
        for reg in regressor_names:
            # Average across blocks and add for each subject
            results_temp[reg] = np.mean(np.array(results_temp[reg]), axis=0)
            results_all[reg] += results_temp[reg]
    return results_all

In [71]:
log_fnames = ["subject_1.hdf5"]
num_blocks = 4
elec_name = "Cz"

# "Null", "Sample_Points"

regressor_names = ["CD_PS_SP", "CD_BS_SP",
                   "CD_PS_AP", "CD_BS_AP",
                   "CD_PS_TP", "CD_BS_TP"]

process_tbt_logs(results_dir, num_blocks, log_fnames, elec_name)

{'CD_PS_SP': array([431.42025276, 458.5129151 , 443.26740976, 473.20519865,
        401.85030701, 436.33478676, 491.05226676, 425.20326699,
        480.46420595, 438.62927756, 485.43132494, 476.54201092,
        463.23289263, 448.70553075, 507.18460498, 450.7187841 ,
        653.26128485, 431.78728557, 400.86212375, 429.97173474]),
 'CD_BS_SP': array([435.35564374, 462.57247792, 448.38568933, 476.91883619,
        405.60025843, 439.76072112, 496.3225068 , 430.3480378 ,
        488.27196711, 444.5389905 , 490.3803854 , 483.1053472 ,
        467.38843565, 455.60436589, 511.73820072, 459.96853178,
        656.21385679, 437.4756284 , 407.0555767 , 435.30712908]),
 'CD_PS_AP': array([431.50908834, 457.50881851, 443.90518916, 471.40957456,
        400.19381668, 434.86153866, 491.89236255, 424.2525848 ,
        477.85391541, 438.05280969, 481.1368684 , 473.63976453,
        463.55636537, 445.34687251, 504.00086887, 454.06687899,
        650.02243672, 428.68467275, 396.61140532, 428.31169637])

In [26]:
h5f = tables.open_file(results_dir + log, mode="a")
h5f.root["subject_0/block_4/elec_C2"]

/subject_0/block_4/elec_C2 (Group) ''
  children := []

In [ ]:
y_labels = ["PS_SP", "BS_SP",
            "PS_AP", "BS_AP",
            "PS_TP", "BS_TP"]

x_labels = ["Block 1", "Block 2", "Block 3", "Block 4"]

heat_data = np.array(list(log_model_evidences.values()))

def heatmap_lme(lme_array, x_labels, y_labels):
    fig, ax = plt.subplots()
    im = ax.imshow(heat_data, cmap="Reds")

    # We want to show all ticks...
    ax.set_xticks(np.arange(len(x_labels)))
    ax.set_yticks(np.arange(len(y_labels)))
    # ... and label them with the respective list entries
    ax.set_xticklabels(x_labels)
    ax.set_yticklabels(y_labels)

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    ax.set_title("Log Model Evidences: FCz")